# Proje

Bu projeyi veribilimi ve makine öğrenmesi alanında öğrendiklerimi kendi üzerimde uygulamak için yapıyorum.

## Amaç

* Verisetini işlenebilir hale getirdikten sonra makineye öğretip; hangi saatlerde daha verimli okuma yaptığımı, hangi saatlerde yazdığım özetin sayfa sayısına oranla daha uzun olduğunu, hangi kitapları beğenip beğenmediğimi, en çok hangi yazarların kitaplarında vakit geçirdiğimi, önceki verilere dayanarak saat bilgisi eksik olan okumalarımın saatlerinin ne olabileceği ve daha birçok soruyu veri bilimi ve makine öğrenmesi kullanarak bilgisayarın cevaplamasını sağlamak.

# Veriseti Hikayesi

Bu veriseti, benim kitap okurken çeşitli verileri defterime kaydedip sonra da excele almama dayanır.

* date = Okumanın tarihi

* start_time = Okumaya başladığım saat

* finish_time = Okumayı bitirdiğim saat

* writer = Okuduğum kitabın yazarı

* book = Kitabın adı

* start_page = Okumaya başladığım sayfanın sayısı

* finish_page = Okumayı bitidiğim sayfanın sayısı

* summary_size = Okuduğum sayfa aralıklarını değerlendirip yazdığım özet ya da çıkarımların kelime sayısı

* is_section_liked = Okuduğum aralığı beğendim mi ? 0=Hayır, 1=Evet



In [66]:
import pandas as pd

In [67]:
my_readings = pd.read_excel("My_readings.xlsx")
df = my_readings.copy()

In [68]:
df.head() #veriseti hakkinda ilk 5 gozlem

,date,start_time,finish_time,writer,book,kind,start_page,finish_page,summary_size,is_section_liked
0,2020-05-03,NaN,NaN,J.P. Sartre,Bulanti,felsefik,15.0,38.0,68.0,1
1,2020-03-09,NaN,NaN,J.P. Sartre,Bulanti,felsefik,39.0,75.0,73.0,1
2,2020-03-11,NaN,NaN,J.P. Sartre,Bulanti,felsefik,75.0,105.0,50.0,1
3,2020-03-11,NaN,NaN,Thomas More,Utopya,utopik,3.0,26.0,91.0,1
4,2020-03-22,NaN,NaN,J.P. Sartre,Bulanti,felsefik,106.0,134.0,27.0,1


In [69]:
df.shape #verisetimizin gözlem ve değişken sayisi

(35, 10)

In [70]:
df.dtypes #verisetimizdeki değişkenlerin türü

date                datetime64[ns]
start_time                  object
finish_time                 object
writer                      object
book                        object
kind                        object
start_page                 float64
finish_page                float64
summary_size               float64
is_section_liked             int64
dtype: object

In [71]:
# is_section_liked değişkenini integerdan booleana çevirelim
df['is_section_liked'] = df['is_section_liked'].astype('bool')

In [72]:
df.dtypes

date                datetime64[ns]
start_time                  object
finish_time                 object
writer                      object
book                        object
kind                        object
start_page                 float64
finish_page                float64
summary_size               float64
is_section_liked              bool
dtype: object

--

#### Değerler arası tip dönüşümlerine devam etmem gerekiyor ancak eksik veri varken yapmıyor, bu sebeple tüm eksik veriler yerine -1 yazacağım, ama saat verisi içerenlere 00:00:00.  böylece onları daha sonra hatırlarım eksik veri olduğunu.

In [73]:
df["start_time"].fillna("00:00:00", inplace = True)
df["finish_time"].fillna("00:00:00", inplace = True)

In [74]:
df.fillna(-1, inplace = True)

In [76]:
df.tail()

,date,start_time,finish_time,writer,book,kind,start_page,finish_page,summary_size,is_section_liked
30,2020-06-15,16:00:00,17:00:00,Dostoyevski,Suc ve Ceza,roman,253.0,296.0,11.0,True
31,2020-06-16,21:00:00,22:00:00,Dostoyevski,Suc ve Ceza,roman,297.0,334.0,26.0,True
32,2020-06-16,22:15:00,22:45:00,Platon,Sokratesin Savunmasi,felsefik,40.0,59.0,31.0,True
33,2020-06-17,16:00:00,16:30:00,Platon,Sokratesin Savunmasi,felsefik,60.0,76.0,40.0,True
34,2020-06-17,21:45:00,22:55:00,Dostoyevski,Suc ve Ceza,roman,335.0,381.0,38.0,True


Eksik değerleri doldurduk, şimdi dönüşümlere devam edebiliriz.

In [77]:
# start_page, finish_page ve summary_size  değişkenlerini integer yapmak istiyorum
df['start_page'] = df['start_page'].astype('int')
df['finish_page'] = df['finish_page'].astype('int')
df['summary_size'] = df['summary_size'].astype('int')
df.dtypes

date                datetime64[ns]
start_time                  object
finish_time                 object
writer                      object
book                        object
kind                        object
start_page                   int64
finish_page                  int64
summary_size                 int64
is_section_liked              bool
dtype: object

--

Şimdi de writer, kind ve book değişkenlerinin kategorik değişken (category) olmasını istiyorum.

In [78]:
#üstte yaptığımızın aksine bu sefer tek tek değil de hepsini bir satır kodda yapalım.

df[['writer', 'kind', 'book']] = df[['writer', 'kind', 'book']].astype('category')
df.dtypes

date                datetime64[ns]
start_time                  object
finish_time                 object
writer                    category
book                      category
kind                      category
start_page                   int64
finish_page                  int64
summary_size                 int64
is_section_liked              bool
dtype: object

In [103]:
# Şimdi de start_time ve finish_time değişkenlerini object'ten datetime'a çevirelim.
import datetime
start_time_obj = df["start_time"]
start_time_obj = start_time_obj.apply((lambda x: datetime.datetime.strptime(x, '%H:%M:%S').time()))
start_time_obj

TypeError: strptime() argument 1 must be str, not datetime.time